# **Preprocessing and Feature Engineering**

### **Imports**
---

In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn import metrics

### **Load the data**
-----

In [236]:
train = pd.read_csv('../data/train_cleaned.csv')
train.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,screen_porch_size,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,SalePrice
0,544,531379050,60,RL,43.0,11492.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,4,2009,WD,220000.0
1,153,535304180,20,RL,68.0,7922.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,1,2010,WD,109000.0
2,318,916386060,60,RL,73.0,9802.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,4,2010,WD,174000.0
3,255,906425045,50,RL,82.0,14235.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,3,2010,WD,138500.0
4,2827,908186070,180,RM,35.0,3675.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,6,2006,New,140000.0


In [237]:
test = pd.read_csv('../data/test_cleaned.csv')
test.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,three_season_porch_size,screen_porch_size,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type
0,2658,902301120,190,RM,69.0,9142.0,Pave,Grvl,Reg,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,9,2006,New
3,1989,902207150,30,RM,60.0,8520.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,7,2007,WD
4,625,535105100,20,RL,NaN,9500.0,Pave,NaN,IR1,Lvl,...,0.0,185.0,0.0,NaN,NaN,NaN,0.0,7,2009,WD


----
### *Redo the data type updates which aren't reflected when reading in file*

In [238]:
train[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts','basement_finished_rating2', 'pool_quality','garage_quality',
       'garage_condition', 'fence_quality', 'misc_features', 'fireplace_quality', 'garage_finished','Garage Type'
      ]] = train[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features','fireplace_quality', 'garage_finished','garage_location']].applymap(lambda itm: 'None' if itm == 'NaN' else str(itm))

test[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features', 'fireplace_quality', 'garage_finished',
       'Garage Type']] = test[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features','fireplace_quality', 'garage_finished','garage_location']].applymap(lambda itm: 'None' if itm == 'NaN' else str(itm))

In [247]:
#to integers
train[['full_bathrooms_basement','half_bathrooms_basement',
      'garage_car_capacity']] = train[['full_bathrooms_basement','half_bathrooms_basement','garage_car_capacity']].astype(int)

test[['full_bathrooms_basement','half_bathrooms_basement',
      'garage_car_capacity']] = test[['full_bathrooms_basement','half_bathrooms_basement','garage_car_capacity']].astype(int)

----
## **Model Attempt 3:** *Running model with all numerical features* (pre-data type update)

In [280]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [281]:
test[features]

,lot_area,house_quality,house_condition,year_built,year_remodeled,finished_area,finished_area2,unfinished_area,total_basement_area,first_fl_area,...,garage_size,wood_deck_size,open_porch_size,enclose_porch_size,three_season_porch_size,screen_porch_size,pool_size,misc_features_value,month_sold,year_sold
0,9142.0,6,8,1910,1950,0,0,1020,1020,908.0,...,440,0.0,60.0,112.0,0.0,0.0,0.0,0.0,4,2006
1,9662.0,5,4,1977,1977,0,0,1967,1967,1967.0,...,580,170.0,0.0,0.0,0.0,0.0,0.0,0.0,8,2006
2,17104.0,7,5,2006,2006,554,0,100,654,664.0,...,426,100.0,24.0,0.0,0.0,0.0,0.0,0.0,9,2006
3,8520.0,5,6,1923,2006,0,0,968,968,968.0,...,480,0.0,0.0,184.0,0.0,0.0,0.0,0.0,7,2007
4,9500.0,6,5,1963,1963,609,0,785,1394,1394.0,...,514,0.0,76.0,0.0,0.0,185.0,0.0,0.0,7,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,8000.0,6,6,1974,1974,931,153,0,1084,1084.0,...,488,0.0,96.0,0.0,0.0,0.0,0.0,0.0,11,2007
874,14670.0,6,7,1966,1999,575,0,529,1104,1104.0,...,480,0.0,230.0,0.0,0.0,0.0,0.0,0.0,8,2008
875,8250.0,5,5,1968,1968,250,492,210,952,1211.0,...,322,0.0,63.0,0.0,0.0,0.0,0.0,0.0,8,2008
876,9000.0,4,6,1971,1971,616,0,248,864,864.0,...,528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2007


In [282]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

Training Shape: ((954, 32), (954,))
Test Shape: ((318, 32), (318,))


In [283]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [284]:
#Training Score
lr.score(X_train, y_train)

0.8798556232216684

In [285]:
#Test Score
lr.score(X_test, y_test)

0.8742177245601109

In [286]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [ ]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

Solid R-Squared results but could be misleading due to the number of features modeled.

In [ ]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

In [ ]:
#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

In [ ]:
#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
rmse

In [ ]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

In [ ]:
# add sale price column to test data set based on model
test['SalePrice'] = preds
test.head()

In [ ]:
# Create Data Frame with only Id and SalePrice
all_numerical_submission = test[['Id', 'SalePrice']]
all_numerical_submission.shape

In [ ]:
# set index for new DF
all_numerical_submission.set_index('Id', inplace=True)
all_numerical_submission.head()

In [ ]:
# save submission
all_numerical_submission.to_csv('../data/all_numerical_submission.csv')

----
## **Model Attempt 4:** *Running model with all numerical features* (data types updated)

In [274]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [275]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

Training Shape: ((954, 32), (954,))
Test Shape: ((318, 32), (318,))


In [276]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [277]:
#Training Score
lr.score(X_train, y_train)

0.8798556232216684

In [278]:
#Test Score
lr.score(X_test, y_test)

0.8742177245601109

In [279]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [ ]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

write some commentary on scores

In [ ]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

In [ ]:
#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

In [ ]:
#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
rmse

Predicted home sale price off by about $18,114 on average relative to observed price. 

In [ ]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

In [ ]:
# add sale price column to test data set based on model
test['SalePrice'] = preds
test.head()

In [ ]:
# Create Data Frame with only Id and SalePrice
all_numerical_submission_types_update = test[['Id', 'SalePrice']]
all_numerical_submission_types_update.shape

In [ ]:
# set index for new DF
all_numerical_submission_types_update.set_index('Id', inplace=True)
all_numerical_submission_types_update.head()

In [ ]:
# save submission
all_numerical_submission_types_update.to_csv('../data/all_numerical_submission_types_update.csv')

------
### *Handling Categorical Variables*

In [ ]:
train.head()

In [ ]:
# changing Y/N and Paved/Gravel Columns to 1's and 0's
train['street_surface'] = train['street_surface'].map({'Pave': 1, 'Grvl': 0})
train['central_air'] = train['central_air'].map({'Y': 1, 'N':0})

test['street_surface'] = test['street_surface'].map({'Pave': 1, 'Grvl': 0})
test['central_air'] = test['central_air'].map({'Y': 1, 'N':0})

train.head()

In [ ]:
# candidates for #'s rather than dummy variables
train['exterior_quality'] = train['exterior_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
train['exterior_condition'] = train['exterior_condition'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['basement_height'] = train['basement_height'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
train['basement_condition'] = train['basement_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['heating_quality'] = train['heating_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['kitchen_quality'] = train['kitchen_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['fireplace_quality'] = train['fireplace_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['garage_quality'] = train['garage_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
train['garage_condition'] = train['garage_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['pool_quality'] = train['pool_quality'].map({np.nan:0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['fence_quality'] = train['fence_quality'].map({np.nan:0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})


test['exterior_quality'] = test['exterior_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
test['exterior_condition'] = test['exterior_condition'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['basement_height'] = test['basement_height'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
test['basement_condition'] = test['basement_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['heating_quality'] = test['heating_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['kitchen_quality'] = test['kitchen_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['fireplace_quality'] = test['fireplace_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['garage_quality'] = test['garage_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
test['garage_condition'] = test['garage_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['pool_quality'] = test['pool_quality'].map({np.nan:0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['fence_quality'] = test['fence_quality'].map({np.nan:0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})

train.head(10)

------
## **Model 5:** *Numbers replacing categorical variables*

In [ ]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [ ]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

lr = LinearRegression()
lr.fit(X_train, y_train)

#Training Score
print(f'Training Score: {lr.score(X_train, y_train)}')
#Test Score
print(f'Test Score: {lr.score(X_test, y_test)}')

In [ ]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

In [ ]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

In [ ]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
print(f'Adjusted R-Squared for training: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
print(f'Adjusted R-Squared for test: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
print(f'Root Mean Squared Error: {rmse}')

Predicted home sale price off by about $17,345 on average relative to observed price. 

In [ ]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

In [ ]:
# add sale price column to test data set based on model
test['SalePrice'] = preds

# Create Data Frame with only Id and SalePrice
categories_converted_numbers = test[['Id', 'SalePrice']]

# set index for new DF
categories_converted_numbers.set_index('Id', inplace=True)
categories_converted_numbers.head()

# save submission
categories_converted_numbers.to_csv('../data/categories_converted_numbers.csv')

-----

### *Binarize the remaining categorical columns*

In [222]:
train = pd.get_dummies(data = train, columns = ['dwelling_type','zoning_type','street_surface','alley_surface','property_shape','property_flatness',
                     'util_avail','lot_position','property_slope','ames_neighborhood','prox_to_transport','prox_to_transport_2',
                     'bldg_type', 'floors','house_quality','house_condition','roof_style', 'roof_material','ext_covering','ext_covering2',
                     'masonry_veneer_type','foundation_type','basement_walkouts','basement_finished_rating','basement_finished_rating2',
                    'heating_type','central_air','electrical_setup', 'full_bathrooms_basement', 'half_bathrooms_basement', 'full_bathrooms_above_ground',
                     'half_bathrooms_above_ground','bedrooms_above_ground', 'kitchens_above_ground', 'home_functionality',
                    'garage_location', 'garage_finished', 'garage_car_capacity','driveway_surface','misc_features', 'sale_type'], drop_first = True)         

test = pd.get_dummies(data = test, columns = ['dwelling_type','zoning_type','street_surface','alley_surface','property_shape','property_flatness',
                     'util_avail','lot_position','property_slope','ames_neighborhood','prox_to_transport','prox_to_transport_2',
                     'bldg_type', 'floors','house_quality','house_condition','roof_style', 'roof_material','ext_covering','ext_covering2',
                     'masonry_veneer_type','foundation_type','basement_walkouts','basement_finished_rating','basement_finished_rating2',
                    'heating_type','central_air','electrical_setup', 'full_bathrooms_basement', 'half_bathrooms_basement', 'full_bathrooms_above_ground',
                     'half_bathrooms_above_ground','bedrooms_above_ground', 'kitchens_above_ground', 'home_functionality',
                    'garage_location', 'garage_finished', 'garage_car_capacity','driveway_surface','misc_features', 'sale_type'], drop_first = True)

KeyError: "None of [Index(['dwelling_type', 'zoning_type', 'street_surface', 'alley_surface',\n       'property_shape', 'property_flatness', 'util_avail', 'lot_position',\n       'property_slope', 'ames_neighborhood', 'prox_to_transport',\n       'prox_to_transport_2', 'bldg_type', 'floors', 'house_quality',\n       'house_condition', 'roof_style', 'roof_material', 'ext_covering',\n       'ext_covering2', 'masonry_veneer_type', 'foundation_type',\n       'basement_walkouts', 'basement_finished_rating',\n       'basement_finished_rating2', 'heating_type', 'central_air',\n       'electrical_setup', 'full_bathrooms_basement',\n       'half_bathrooms_basement', 'full_bathrooms_above_ground',\n       'half_bathrooms_above_ground', 'bedrooms_above_ground',\n       'kitchens_above_ground', 'home_functionality', 'garage_location',\n       'garage_finished', 'garage_car_capacity', 'driveway_surface',\n       'misc_features', 'sale_type'],\n      dtype='object')] are in the [columns]"

In [211]:
print(train.shape)
train.head(1)

(1272, 245)


,Id,parcel_id,lot_frontage,lot_area,year_built,year_remodeled,masonry_veneer_area,exterior_quality,exterior_condition,basement_height,...,misc_features_Shed,misc_features_nan,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,544,531379050,43.0,11492.0,1996,1997,132.0,3,2,4,...,0,1,0,0,0,0,0,0,0,1


------
## **Model 6**: Binarized all remaining categorical columms

In [212]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

lr = LinearRegression()
lr.fit(X_train, y_train)

#Training Score
print(f'Training Score: {lr.score(X_train, y_train)}')
#Test Score
print(f'Test Score: {lr.score(X_test, y_test)}')

KeyError: "['zoning_type_I (all)', 'util_avail_NoSewr', 'ames_neighborhood_BrDale', 'prox_to_transport_2_PosA', 'floors_2.5Fin', 'roof_material_Metal', 'roof_material_Roll', 'roof_material_WdShngl', 'ext_covering_AsphShn', 'ext_covering_PreCast', 'ext_covering2_Other', 'ext_covering2_PreCast', 'masonry_veneer_type_CBlock', 'masonry_veneer_type_nan', 'foundation_type_Slab', 'basement_finished_rating_nan', 'basement_finished_rating2_nan', 'heating_type_GasA', 'full_bathrooms_above_ground_4', 'kitchens_above_ground_1', 'garage_finished_nan', 'garage_car_capacity_1', 'sale_type_VWD'] not in index"

In [ ]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

In [ ]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

In [ ]:
# add sale price column to test data set based on model
test['SalePrice'] = preds

# Create Data Frame with only Id and SalePrice
all_features_binarized = test[['Id', 'SalePrice']]

# set index for new DF
all_features_binarized.set_index('Id', inplace=True)
all_features_binarized.head()

# save submission
all_features_binarized.to_csv('../data/all_features_binarized.csv')

------

In [ ]:
# candidates for interaction features or dropping collinear?
house & exterior & garage - - quality/condition
'basement_height', 'basement_condition'

# collinear?
# check with correlation/plot if 90 or greater, drop 1 of them that's less correlated with price
droppping one of the quality / condition ones

In [ ]:
# comparing correlations of all categorical features to target 
train.corr()[['sale_price']].sort_values(by = 'sale_price', ascending=False)

In [9]:
# at this point save a version to model with to try all features
# also save model after before each (combining, interaction, dropping) - run all features through after each to see if any improvement

------
### *Combining Features*

In [8]:
# combine bedrooms
# combine bathrooms
# combine kitchens (above and below ground)
# combine area of home (finished below and above)

### *Interaction Terms*

### *Dropping Collinear Features*

In [ ]:
# for exploration and modelling
#1) run model where all features includes - X = df.drop(columns = 'SalesPrice')

# Three different way to create X:
#X = ads[['TV', 'radio', 'newspaper']]

#X = ads.drop(columns = ['sales']) # recommended with high dimensional datasets

model without feature engineering, interaction terms, dropping collinear features manually for all variables